# The Annotated Transformer 보다 친절한 트랜스포머 튜토리얼 !

## 0. 데이터 가공
- 먼저, [**AI Hub**](http://www.aihub.or.kr/)에서 [**한국어-영어 번역 말뭉치**](http://www.aihub.or.kr/aidata/87)를 요청해야 합니다.
- 신청 후, 약 **2일** 내에 승인 결과가 메일로 전달된다고 합니다.

In [69]:
import pandas as pd
data = pd.read_csv('data/corpus.csv', encoding='utf-8')

In [72]:
# 한국어, 영어 데이터 별개로 저장

kor_lines = []
eng_lines = []

for _, row in data.iterrows():
    _, kor, eng = row
    kor_lines.append(kor)
    eng_lines.append(eng)

In [76]:
for kor, eng in zip(kor_lines[:5], eng_lines[:5]):
    print(f'[KOR]: {kor}')
    print(f'[ENG]: {eng}\n')

[KOR]: 나는 매일 저녁 배트를 만나러 다락방으로 가요.
[ENG]: I go to the attic every evening to meet Bat.

[KOR]: 선생님 이문장이 이해가 안 가요.
[ENG]: Sir, I don't understand this sentence here.

[KOR]: 컴퓨터를 시작하면 시간이 너무 빠르게 가요.
[ENG]: Time flies when you start using the computer.

[KOR]: 나는 오늘 자정에 한국으로 돌아 가요.
[ENG]: I'm going back to Korea today at midnight.

[KOR]: 나는 일어나자마자 화장실에 가요.
[ENG]: I go to bathroom as soon as I wake up.



In [80]:
with open('train_korean.txt', 'w', encoding='utf-8') as f:
    for line in kor_lines:
        print(line, file=f)
        
with open('train_english.txt', 'w', encoding='utf-8') as f:
    for line in eng_lines:
        print(line, file=f)

## 1. BPE 토크나이저 학습
- 앞서 가공한 데이터를 활용해 **BPE 토크나이저**를 학습시킵니다.

In [39]:
params = {
    'batch_size': 64,
    'num_epoch': 15,
    'dropout': 0.1,
    'min_frequency': 3,
    'max_len': 512,
    
    'vocab_size': 20000,
    'num_layers': 6,
    'num_heads': 8,
    'hidden_dim': 512,
    'ffn_dim': 2048,
}

In [81]:
from tokenizers import BPETokenizer

# 한국어 토크나이저 초기화

kor_tokenizer = BPETokenizer()


# 한국어 토크나이저 훈련

kor_tokenizer.train(
    ['train_korean.txt'],
    vocab_size=params['vocab_size'],
    min_frequency=params['min_frequency'],
    special_tokens=['[PAD]', '[SOS]', '[EOS]', '[UNK]'],
    suffix=''
)

In [87]:
# 영어 토크나이저 초기화

eng_tokenizer = BPETokenizer()


# 영어 토크나이저 훈련

eng_tokenizer.train(
    ['train_english.txt'],
    vocab_size=params['vocab_size'],
    min_frequency=params['min_frequency'],
    special_tokens=['[PAD]', '[SOS]', '[EOS]', '[UNK]'],
    suffix=''
)

In [83]:
kor_tokenizer._parameters

{'model': 'BPE', 'unk_token': '<unk>', 'suffix': '</w>', 'dropout': None}

In [ ]:
eng_tokenizer._parameters

In [88]:
pad_idx = kor_tokenizer.token_to_id('[PAD]')
sos_idx = kor_tokenizer.token_to_id('[SOS]')
eos_idx = kor_tokenizer.token_to_id('[EOS]')

In [89]:
kor_tokenizer.enable_padding(pad_id=pad_idx, pad_token='[PAD]', max_length=params['max_len'])
eng_tokenizer.enable_padding(pad_id=pad_idx, pad_token='[PAD]', max_length=params['max_len'])

In [90]:
kor_encoded_data = kor_tokenizer.encode_batch(kor_lines)
eng_encoded_data = eng_tokenizer.encode_batch(eng_lines)

In [91]:
def postprocess(input_ids):
    input_ids = [sos_idx] + input_ids
    
    input_ids = input_ids[:params['max_len']]

    if pad_idx in input_ids:
        pad_start = input_ids.index(pad_idx)
        input_ids[pad_start] = eos_idx
    else:
        input_ids[-1] = eos_idx
    
    return input_ids

In [92]:
kor_processed_data = [postprocess(data.ids) for data in kor_encoded_data]
eng_processed_data = [postprocess(data.ids) for data in eng_encoded_data]

In [121]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random

random.seed(32)
np.random.seed(32)
torch.manual_seed(32)
torch.cuda.manual_seed(32)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torch.utils import data

kor_tensors = [torch.LongTensor(line).to(device) for line in kor_processed_data]
eng_tensors = [torch.LongTensor(line).to(device) for line in eng_processed_data]

src_iter = data.DataLoader(kor_tensors, batch_size=params['batch_size'], shuffle=True, worker_init_fn=np.random.seed(12))
tgt_iter = data.DataLoader(eng_tensors, batch_size=params['batch_size'], shuffle=True, worker_init_fn=np.random.seed(12))

In [124]:
for src, tgt in zip(src_iter, tgt_iter):
    src = src.detach().cpu().numpy()
    tgt = tgt.detach().cpu().numpy()
    print(f'Source: {kor_tokenizer.decode(src[0])}')
    print(f'Target: {eng_tokenizer.decode(tgt[0])}\n')
    break

Source: 몇시에도착할건지말해줄래요?
Target: inthattime,damagedcasewasalreadyexcluded.



## 2. Transformer 모델 구현

- 먼저 모델 구현에 필요한 라이브러리들을 모두 임포트합니다.
- 실험을 함에 있어 항상 실험의 **Reproducibility**를 보장하기 위해 Seed 설정을 해줍니다.

In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

torch.manual_seed(32)
torch.cuda.manual_seed(32)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [168]:
class MultiHeadAttention(nn.Module):
    '''멀티 헤드 어텐션 레이어'''
    def __init__(self, params):
        super(MultiHeadAttention, self).__init__()
        assert params['hidden_dim'] % params['num_heads'] == 0, "hidden dimension must be divisible by the number of heads"
        self.num_heads = params['num_heads']
        self.attn_dim = params['hidden_dim'] // self.num_heads
        
        self.q_w = nn.Linear(params['hidden_dim'], self.num_heads * self.attn_dim)
        self.k_w = nn.Linear(params['hidden_dim'], self.num_heads * self.attn_dim)
        self.v_w = nn.Linear(params['hidden_dim'], self.num_heads * self.attn_dim)
        
        self.o_w = nn.Linear(self.num_heads * self.attn_dim, params['hidden_dim'])
        
    def forward(self, q, k, v, mask=None):
        " q, k, v = [배치 사이즈, 문장 길이, 은닉 차원] "
        
        batch_size = q.size(0)
        
        q = self.q_w(q).view(batch_size, -1, self.num_heads, self.attn_dim).transpose(1, 2)
        k = self.k_w(k).view(batch_size, -1, self.num_heads, self.attn_dim).transpose(1, 2)
        v = self.v_w(v).view(batch_size, -1, self.num_heads, self.attn_dim).transpose(1, 2)
        # q, k, v = [배치 사이즈, 헤드 갯수, 문장 길이, 어텐션 차원]
        
        attn = torch.matmul(q, k.transpose(-1, -2))
        # attn = [배치 사이즈, 헤드 갯수, 문장 길이, 문장 길이]
        
        if mask is not None:
            mask = mask.unsqueeze(1)
            attn.masked_fill(mask==0, -1e9)
        
        score = F.softmax(attn, dim=-1)
        # score = [배치 사이즈, 헤드 갯수, 문장 길이, 문장 길이]
        
        output = torch.matmul(score, v)
        # output = [배치 사이즈, 헤드 갯수, 문장 길이, 어텐션 차원]
        
        output = output.transpose(1, 2).contiguous()
        # output = [배치 사이즈, 문장 길이, 헤드 갯수, 어텐션 차원]
        
        output = output.view(batch_size, -1, self.num_heads * self.attn_dim)
        # output = [배치 사이즈, 문장 길이, 은닉 차원]
        
        output = self.o_w(output)
        # output = [배치 사이즈, 문장 길이, 은닉 차원]
        
        return output, score

In [169]:
def create_src_mask(src):
    " source = [배치 사이즈, 소스 문장 길이] "

    src_len = src.size(1)
    
    src_mask = (src == pad_idx)
    # src_mask = [배치 사이즈, 소스 문장 길이]
    
    src_mask = src_mask.unsqueeze(1).repeat(1, src_len, 1)
    # src_mask = [배치 사이즈, 소스 문장 길이, 소스 문장 길이]
    
    return src_mask


def create_tgt_mask(src, tgt):
    " src = [배치 사이즈, 소스 문장 길이] "
    " tgt = [배치 사이즈, 타겟 문장 길이] "

    batch_size, tgt_len = tgt.size()
    
    subsequent_mask = torch.triu(torch.ones(tgt_len, tgt_len), diagonal=1)
    # subsequent_mask = [타겟 문장 길이, 타겟 문장 길이]
    
    subsequent_mask = subsequent_mask.unsqueeze(0).repeat(batch_size, 1, 1)
    # subsquent_mask = [배치 사이즈, 타겟 문장 길이, 타겟 문장 길이]
    
    src_mask = (src == pad_idx)
    tgt_mask = (tgt == pad_idx)
    # src_mask = [배치 사이즈, 소스 문장 길이]
    # tgt_mask = [배치 사이즈, 타겟 문장 길이]
    
    src_mask = src_mask.unsqueeze(1).repeat(1, tgt_len, 1)
    tgt_mask = tgt_mask.unsqueeze(1).repeat(1, tgt_len, 1)
    # src_mask = [배치 사이즈, 타겟 문장 길이, 소스 문장 길이]
    # tgt_mask = [배치 사이즈, 타겟 문장 길이, 타겟 문장 길이]
    
    tgt_mask = target_mask | subsequent_mask
    
    return src_mask, tgt_mask

![](img/positionwise.png)

In [170]:
class PositionwiseFeedForward(nn.Module):
    '''포지션 와이즈 피드 포워드 레이어'''
    def __init__(self, parmas):
        super(PositionwiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(params['hidden_dim'], params['ffn_dim'])
        self.fc2 = nn.Linear(params['ffn_dim'], params['hidden_dim'])
        self.dropout = nn.Dropout(params['dropout'])
    
    def forward(self, x):
        " x = [배치 사이즈, 문장 길이, 은닉 차원] "

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

![](img/pos.png)

In [171]:
class PositionalEncoding(nn.Module):
    def __init__(self, params):
        super(PositionalEncoding, self).__init__()
        sinusoid = np.array([pos / np.power(10000, 2 * i / params['hidden_dim'])
                            for pos in range(params['max_len']) for i in range(params['hidden_dim'])])
        # sinusoid = [문장 최대 길이 * 은닉 차원]

        sinusoid = sinusoid.reshape(params['max_len'], -1)
        # sinusoid = [문장 최대 길이, 은닉 차원]

        sinusoid[:, 0::2] = np.sin(sinusoid[:, 0::2])
        sinusoid[:, 1::2] = np.cos(sinusoid[:, 1::2])
        sinusoid = torch.FloatTensor(sinusoid).to(device)

        self.embedding = nn.Embedding.from_pretrained(sinusoid, freeze=True)
        
    def forward(self, x):
        " x = [배치 사이즈, 문장 길이] "
        
        pos = torch.arange(x.size(-1), dtype=torch.long).to(device)
        # pos = [배치 사이즈, 문장 길이]

        embed = self.embedding(pos)
        # embed = [배치 사이즈, 문장 길이, 은닉 차원]
        return embed

![](img/encoder.png)

In [172]:
class EncoderLayer(nn.Module):
    '''인코더 레이어'''
    def __init__(self, params):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(params)
        self.layer_norm1 = nn.LayerNorm(params['hidden_dim'])
        self.feed_forward = PositionwiseFeedForward(params)
        self.layer_norm2 = nn.LayerNorm(params['hidden_dim'])
        self.dropout = nn.Dropout(params['dropout'])
        
    def forward(self, x, src_mask):
        " x = [배치 사이즈, 문장 길이, 은닉 차원] "
        
        residual = x
        x = self.self_attn(x, x, x, src_mask)
        x = self.dropout(x)
        x = residual + x
        x = self.layer_norm1(x)
        
        residual = x
        x = self.feed_forward(x)
        x = self.dropout(x)
        x = resiudal + x
        x = self.layer_norm2(x)
        
        return x


class Encoder(nn.Module):
    '''트랜스포머 인코더'''
    def __init__(self, params):
        super(Encoder, self).__init__()
        self.tok_embedding = nn.Embedding(params['vocab_size'], params['hidden_dim'], padding_idx=pad_idx)
        self.pos_embedding = PositionalEncoding(params)
        self.layers = nn.ModuleList([EncoderLayer(params) for _ in range(params['num_layers'])])
        
    def forward(self, src):
        " src = [배치 사이즈, 소스 문장 길이] "

        src_mask = create_src_mask(src)
        src = self.tok_embedding(src) + self.pos_embedding(src)
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        # src = [배치 사이즈, 소스 문장 길이, 은닉 차원]
        return src

![](img/decoder.png)

In [173]:
class DecoderLayer(nn.Module):
    '''디코더 레이어'''
    def __init__(self, params):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(params)
        self.layer_norm1 = nn.LayerNorm(params['hidden_dim'])

        self.enc_dec_attn = MultiHeadAttention(params)
        self.layer_norm2 = nn.LayerNorm(params['hidden_dim'])
        
        self.feed_forward = PositionwiseFeedForward(params)
        self.layer_norm3 = nn.LayerNorm(params['hidden_dim'])
        
        self.dropout = nn.Dropout(params['dropout'])
        
    def forward(self, x, tgt_mask, enc_output, src_mask):
        " x = [배치 사이즈, 문장 길이, 은닉 차원] "
        
        residual = x
        x = self.self_attn(x, x, x, tgt_mask)
        x = self.dropout(x)
        x = residual + x
        x = self.layer_norm1(x)
        
        residual = x
        x = self.enc_dec_attn(x, enc_output, enc_output, src_mask)
        x = self.dropout(x)
        x = residual + x
        x = self.layer_norm2(x)
        
        residual = x
        x = self.feed_forward(x)
        x = self.dropout(x)
        x = resiudal + x
        x = self.layer_norm3(x)
        
        return x


class Decoder(nn.Module):
    '''트랜스포머 디코더'''
    def __init__(self, params):
        super(Decoder, self).__init__()
        self.tok_embedding = nn.Embedding(params['vocab_size'], params['hidden_dim'], padding_idx=pad_idx)
        self.pos_embedding = PositionalEncoding(params)
        self.layers = nn.ModuleList([DecoderLayer(params) for _ in range(params['num_layers'])])
        
    def forward(self, tgt, enc_out):
        " tgt = [배치 사이즈, 타겟 문장 길이] "

        src_mask, tgt_mask = create_tgt_mask(enc_out, tgt)
        tgt = self.tok_embedding(tgt) + self.pos_embedding(tgt)
        
        for layer in self.layers:
            tgt, attn_map = layer(tgt, tgt_mask, enc_out, src_mask)
            
        tgt = torch.matmul(tgt, self.tok_embedding.weight.transpose(0, 1))
        # tgt = [배치 사이즈, 타겟 문장 길이, 은닉 차원]

        return tgt, attn_map

In [174]:
class Transformer(nn.Module):
    '''트랜스포머 네트워크'''
    def __init__(self, params):
        super(Transformer, self).__init__()
        self.encoder = Encoder(params)
        self.decoder = Decoder(params)
    
    def forward(self, src, tgt):
        " src = [배치 사이즈, 소스 문장 길이] "
        " tgt = [배치 사이즈, 타겟 문장 길이] "
        
        enc_out = self.encoder(src)
        dec_out, attn = self.decoder(tgt, enc_out)
        return dec_out, attn
    
    def count_params(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

![](img/optim.png)

In [175]:
class ScheduledOptim:
    '''스케줄 옵티마이저'''
    def __init__(self, optimizer, warmup_steps):
        self.init_lr = np.power(params['hidden_dim'], -0.5)
        self.optimizer = optimizer
        self.step_num = 0
        self.warmup_steps = warmup_steps
    
    def step(self):
        self.step_num += 1
        lr = self.init_lr * self.get_scale()
        
        for p in self.optimizer.param_gropus:
            p['lr'] = lr
            
        self.optimizer.step()
    
    def zero_grad(self):
        self.optimizer.zero_grad()
    
    def get_scale(self):
        return np.min([
            np.power(self.step_num, -0.5),
            self.step_num * np.power(self.warmup_steps, -1.5)
        ])

## 3. 모델 학습

In [176]:
model = Transformer(params)
model.to(device)
model.count_params()

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
criterion.to(device)

optimizer = ScheduledOptim(
    optim.Adam(model.parameters(), betas=[0.9, 0.98], eps=1e-9),
    warmup_steps=4000
)

for epoch in range(params['num_epoch']):
    model.train()
    epoch_loss = 0

    for src, tgt in zip(src_iter, tgt_iter):
        " src = [배치 사이즈, 소스 문장 길이] "
        " tgt = [배치 사이즈, 타겟 문장 길이] "
        
        optimizer.zero_grad()
        
        logits, _ = model(src, tgt[:, :-1])
        # logits = [배치 사이즈, 타겟 문장 길이, 은닉 차원]
        
        logits = logits.contiguous().view(-1, logits.size(-1))
        # logits = [(배치 사이즈 * 타겟 문장 길이) - 1, 은닉 차원]
        golds = target[:, 1:].contiguous.view(-1)
        # golds = [(배치 사이즈 * 타겟 문장 길이) - 1]

        loss = criterion(logits, golds)
        epoch_loss += loss.item()
        
        loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), self.params.clip)
        optimizer.step()
        
    print()

torch.Size([64, 8, 512, 512])
torch.Size([64, 512, 512])


RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 6.00 GiB total capacity; 4.32 GiB already allocated; 369.14 MiB free; 12.56 MiB cached)

## 4. 결과 확인

In [16]:
def predict():
    model.eval()

### 참고자료
- [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)
- [jadore801120/attention-is-all-you-need-pytorch](https://github.com/jadore801120/attention-is-all-you-need-pytorch)
- [tunz/transformer-pytorch](https://github.com/tunz/transformer-pytorch)
- [IgorSusmelj/pytorch-styleguide](https://github.com/IgorSusmelj/pytorch-styleguide)


### TODO
- **Beam Search** 디코딩 추가
- **Label Smoothing** 기법 추가